In [1]:
import pandas as pd
import numpy as np

MI = pd.read_csv('MI.csv',header=2)
MI

,Obs #,Size,Volatility,POV,LnMktCap,Cost_bp
0,1,0.12,0.37,0.36,16.48,115.41
1,2,0.01,0.32,0.04,15.09,9.06
2,3,0.08,0.46,0.22,21.70,81.57
3,4,0.21,0.37,0.36,28.53,139.22
4,5,0.11,0.43,0.09,20.22,43.22
5,6,0.32,0.46,0.39,16.71,124.46
6,7,0.31,0.31,0.57,27.04,158.34
7,8,0.04,0.38,0.23,28.79,58.73
8,9,0.18,0.33,0.17,15.23,61.80
9,10,0.21,0.32,0.33,17.26,116.21


In [2]:
MI.shape

(2500, 6)

In [3]:
r = np.random.RandomState(1234)
MI_s = MI.sample(n=10,replace =True,random_state=r)
MI_s

,Obs #,Size,Volatility,POV,LnMktCap,Cost_bp
723,724,0.18,0.34,0.29,27.80,62.40
1318,1319,0.12,0.38,0.43,16.99,76.90
664,665,0.23,0.37,0.25,26.74,110.49
2041,2042,0.24,0.41,0.20,18.05,98.99
279,280,0.14,0.48,0.09,22.12,47.65
1257,1258,0.07,0.39,0.23,18.73,64.51
30,31,0.09,0.35,0.10,24.02,27.83
1182,1183,0.10,0.34,0.25,21.62,72.65
1398,1399,0.12,0.45,0.15,17.67,67.15
2490,2491,0.34,0.42,0.47,22.10,130.61


# minimize function

In [4]:
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
%matplotlib inline

def solve(sample):
    Y = sample['Cost_bp'].values
    size = sample['Size'].values
    volatility = sample['Volatility'].values
    POV = sample['POV'].values
    LnMktCap = sample['LnMktCap'].values
    xdata = np.array([size,volatility,POV, LnMktCap])
    #print xdata[0]
    #print xdata

    def model(x, coeffs):
        s = np.array(x[0])
        v = np.array(x[1])
        P = np.array(x[2])
        L = np.array(x[3])
   
        return coeffs[0]*(s**coeffs[1])*(v **coeffs[2])*(L**coeffs[4]) * (coeffs[5]*(P **coeffs[3])+ (1-coeffs[5]))

        #coeffs[0] is a1
        #coeffs[1] is a2
        #coeffs[2] is a3
        #coeffs[3] is a4
        #coeffs[4] is a5
        #coeffs[5] is b1

    x0 = np.array([500,0.5,0.5,0.5,0.5,0.5], dtype=float)  # An approximative initial solution that we can find from looking at the graph is for instance:

    def residuals(coeffs,x ,y):
        return model(x, coeffs)-y

    result = least_squares(residuals, x0, bounds=([0,0,0,0,0,0],[2000,1,1,1,1,1]), args=(xdata,Y))
    #print result.x, result.cost, result.optimality
    #print result.x
    #print Y
    #print  model(xdata, result.x )
    #plt.plot(Y,"o--", model(xdata, result.x ), '-')
    return result.x



print solve(MI_s)
#solve(MI_s)

[  3.92403911e+02   4.21249117e-01   4.18507717e-01   3.15791115e-01
   3.76588524e-11   1.00000000e+00]


# with replacement

In [5]:
df = pd.DataFrame(columns=['a1','a2','a3','a4','a5','b1'])
for i in xrange(250):
    MI_s = MI.sample(n=250,replace =True)
    result = pd.DataFrame(np.reshape(solve(MI_s),(1,6)),columns=['a1','a2','a3','a4','a5','b1'])
    df = df.append(result)
df = df.reset_index(drop=True)
df

,a1,a2,a3,a4,a5,b1
0,739.442784,0.359189,0.668675,0.660144,6.635074e-16,1.000000
1,548.239978,0.356576,0.504823,0.784902,8.051496e-02,0.977216
2,497.491152,0.320838,0.470475,0.651040,4.329757e-02,1.000000
3,600.524911,0.334584,0.568966,0.802610,5.675275e-02,0.945969
4,804.177974,0.346098,0.576591,1.000000,1.712920e-17,0.902239
5,611.575133,0.336613,0.490175,0.658896,2.464880e-12,1.000000
6,482.220396,0.311219,0.517831,0.682909,7.507930e-02,1.000000
7,707.171376,0.345590,0.626821,0.662960,2.384308e-03,1.000000
8,690.578350,0.332216,0.577740,0.689687,7.371842e-23,1.000000
9,709.736478,0.363691,0.564333,0.689218,1.109211e-13,1.000000


In [6]:
summary = df.describe()
se =  summary.ix['std'].values/summary.ix['count'].values
se = pd.DataFrame(np.reshape(se,(1,6)),index=['se'],columns=['a1','a2','a3','a4','a5','b1'])
summary = summary.append(se)
t = summary.ix['mean'].values/summary.ix['se'].values
t = pd.DataFrame(np.reshape(t,(1,6)),index=['t'],columns=['a1','a2','a3','a4','a5','b1'])
summary = summary.append(t)
summary

,a1,a2,a3,a4,a5,b1
count,250.000000,250.000000,250.000000,250.000000,2.500000e+02,250.000000
mean,644.183221,0.343762,0.552949,0.749400,2.730521e-02,0.973770
std,110.819824,0.019852,0.094453,0.092462,4.048615e-02,0.031490
min,334.537126,0.292435,0.294841,0.617331,2.779702e-28,0.886940
25%,575.874738,0.329310,0.484593,0.680149,3.662523e-14,0.952085
50%,652.790022,0.343569,0.551632,0.715957,1.953450e-10,0.987707
75%,715.303307,0.357667,0.614782,0.800485,4.587750e-02,1.000000
max,970.635573,0.388543,0.867533,1.000000,2.031069e-01,1.000000
se,0.443279,0.000079,0.000378,0.000370,1.619446e-04,0.000126
t,1453.221989,4329.068741,1463.551044,2026.236534,1.686083e+02,7730.827104


# without replacement

In [7]:
df2 = pd.DataFrame(columns=['a1','a2','a3','a4','a5','b1'])
for i in xrange(250):
    MI_s = MI.sample(n=250,replace =False)
    result = pd.DataFrame(np.reshape(solve(MI_s),(1,6)),columns=['a1','a2','a3','a4','a5','b1'])
    df2 = df2.append(result)
df2 = df2.reset_index(drop=True)
df2

,a1,a2,a3,a4,a5,b1
0,619.078578,0.350391,0.422905,0.698400,7.338197e-11,1.000000
1,724.911717,0.358706,0.596715,0.713948,3.919126e-12,0.967813
2,702.884192,0.375588,0.541333,0.672964,1.724529e-11,1.000000
3,724.919657,0.320768,0.625126,0.792058,2.472793e-07,0.959769
4,738.370300,0.356481,0.685689,0.753439,2.976135e-02,0.973873
5,786.443827,0.337584,0.736212,0.702414,1.259607e-08,0.992280
6,591.899821,0.343893,0.555460,0.951643,8.076229e-02,0.911826
7,612.326318,0.310496,0.495514,0.919404,1.450198e-02,0.901326
8,603.633177,0.357519,0.471838,0.712027,3.124402e-02,1.000000
9,617.704275,0.337385,0.535319,0.703409,3.219802e-02,1.000000


In [8]:
summary2 = df2.describe()
se =  summary2.ix['std'].values/summary2.ix['count'].values
se = pd.DataFrame(np.reshape(se,(1,6)),index=['se'],columns=['a1','a2','a3','a4','a5','b1'])
summary2 = summary2.append(se)
t = summary2.ix['mean'].values/summary2.ix['se'].values
t = pd.DataFrame(np.reshape(t,(1,6)),index=['t'],columns=['a1','a2','a3','a4','a5','b1'])
summary2 = summary2.append(t)
summary2

,a1,a2,a3,a4,a5,b1
count,250.000000,250.000000,250.000000,250.000000,2.500000e+02,250.000000
mean,650.492792,0.343453,0.560969,0.750794,2.642309e-02,0.974180
std,96.284854,0.017242,0.078531,0.085129,3.901102e-02,0.030294
min,361.749661,0.297167,0.368279,0.618483,1.488919e-28,0.882976
25%,596.257057,0.331662,0.507380,0.685677,3.245056e-13,0.951855
50%,657.440407,0.342941,0.553750,0.721763,1.299377e-07,0.986727
75%,718.101209,0.356312,0.618999,0.802358,4.455241e-02,1.000000
max,896.488564,0.406227,0.760962,1.000000,1.613690e-01,1.000000
se,0.385139,0.000069,0.000314,0.000341,1.560441e-04,0.000121
t,1688.980047,4979.800449,1785.825903,2204.876108,1.693309e+02,8039.427129


# a5(LnMktCap) is insignificant, new function without LnMktCap

In [9]:

from scipy.optimize import least_squares
import matplotlib.pyplot as plt
%matplotlib inline

def solve(sample):
    Y = sample['Cost_bp'].values
    size = sample['Size'].values
    volatility = sample['Volatility'].values
    POV = sample['POV'].values
    
    xdata = np.array([size,volatility,POV])
    #print xdata[0]
    #print xdata

    def model(x, coeffs):
        s = np.array(x[0])
        v = np.array(x[1])
        P = np.array(x[2])
        
   
        return coeffs[0]*(s**coeffs[1])*(v **coeffs[2]) * (coeffs[4]*(P **coeffs[3])+ (1-coeffs[4]))

        #coeffs[0] is a1
        #coeffs[1] is a2
        #coeffs[2] is a3
        #coeffs[3] is a4
        #coeffs[4] is b1


    x0 = np.array([500,0.5,0.5,0.5,0.5], dtype=float)  # An approximative initial solution that we can find from looking at the graph is for instance:

    def residuals(coeffs,x ,y):
        return model(x, coeffs)-y

    result = least_squares(residuals, x0, bounds=([0,0,0,0,0],[2000,1,1,1,1]), args=(xdata,Y))
    #print result.x, result.cost, result.optimality
    #print result.x
    #print Y
    #print  model(xdata, result.x )
    #plt.plot(Y,"o--", model(xdata, result.x ), '-')
    return result.x



print solve(MI_s)
#solve(MI_s)

[  7.66977445e+02   3.59246652e-01   6.16647022e-01   7.40387242e-01
   9.85972684e-01]


# with replacement without LnMktCap

In [10]:
df = pd.DataFrame(columns=['a1','a2','a3','a4','b1'])
for i in xrange(250):
    MI_s = MI.sample(n=250,replace =True)
    result = pd.DataFrame(np.reshape(solve(MI_s),(1,5)),columns=['a1','a2','a3','a4','b1'])
    df = df.append(result)
df = df.reset_index(drop=True)
df

,a1,a2,a3,a4,b1
0,687.559430,0.328946,0.542986,0.814278,0.947333
1,864.421838,0.356129,0.700420,0.957372,0.901772
2,686.341209,0.335010,0.579315,0.739954,0.977327
3,810.366009,0.361753,0.649808,0.816652,0.960405
4,726.968168,0.346494,0.690768,0.628208,1.000000
5,781.271024,0.348193,0.681808,0.711557,1.000000
6,662.041861,0.349374,0.502517,0.692929,1.000000
7,840.881042,0.352713,0.673622,1.000000,0.878188
8,637.831669,0.349056,0.514346,0.651307,1.000000
9,726.217676,0.328171,0.555445,0.932716,0.916538


In [11]:
summary = df.describe()
se =  summary.ix['std'].values/summary.ix['count'].values
se = pd.DataFrame(np.reshape(se,(1,5)),index=['se'],columns=['a1','a2','a3','a4','b1'])
summary = summary.append(se)
t = summary.ix['mean'].values/summary.ix['se'].values
t = pd.DataFrame(np.reshape(t,(1,5)),index=['t'],columns=['a1','a2','a3','a4','b1'])
summary = summary.append(t)
summary

,a1,a2,a3,a4,b1
count,250.000000,250.000000,250.000000,250.000000,250.000000
mean,703.983443,0.343100,0.560260,0.759712,0.971267
std,83.562796,0.019509,0.092675,0.102124,0.034373
min,546.386735,0.287501,0.335280,0.628208,0.872731
25%,643.515950,0.330392,0.492811,0.681266,0.944765
50%,696.357066,0.342548,0.557578,0.717779,0.987898
75%,757.392185,0.355325,0.621516,0.823077,1.000000
max,935.898183,0.401564,0.839446,1.000000,1.000000
se,0.334251,0.000078,0.000371,0.000408,0.000137
t,2106.150936,4396.671486,1511.360611,1859.778912,7064.210004


# without replacement without LnMktCap

In [12]:
df2 = pd.DataFrame(columns=['a1','a2','a3','a4','b1'])
for i in xrange(250):
    MI_s = MI.sample(n=250,replace =False)
    result = pd.DataFrame(np.reshape(solve(MI_s),(1,5)),columns=['a1','a2','a3','a4','b1'])
    df2 = df2.append(result)
df2 = df2.reset_index(drop=True)
df2

,a1,a2,a3,a4,b1
0,674.137673,0.318814,0.558389,0.780546,0.942595
1,669.443088,0.293902,0.623515,0.707813,1.000000
2,722.636339,0.347382,0.625602,0.688748,1.000000
3,684.254907,0.346851,0.543260,0.687068,0.995869
4,551.961963,0.332910,0.331937,0.745995,0.965693
5,777.064890,0.370087,0.621534,0.779726,0.960805
6,677.825638,0.359336,0.511240,0.696949,1.000000
7,692.588820,0.337128,0.633604,0.668443,1.000000
8,642.428745,0.348949,0.481510,0.678481,1.000000
9,743.519377,0.338648,0.616844,0.812213,0.952382


In [13]:
summary2 = df2.describe()
se =  summary2.ix['std'].values/summary2.ix['count'].values
se = pd.DataFrame(np.reshape(se,(1,5)),index=['se'],columns=['a1','a2','a3','a4','b1'])
summary2 = summary2.append(se)
t = summary2.ix['mean'].values/summary2.ix['se'].values
t = pd.DataFrame(np.reshape(t,(1,5)),index=['t'],columns=['a1','a2','a3','a4','b1'])
summary2 = summary2.append(t)
summary2

,a1,a2,a3,a4,b1
count,250.000000,250.000000,250.000000,250.000000,250.000000
mean,708.584136,0.343016,0.571493,0.755385,0.972251
std,80.977429,0.018423,0.092520,0.094720,0.032141
min,533.170464,0.293902,0.318603,0.610409,0.884820
25%,655.035446,0.332462,0.508028,0.681575,0.949497
50%,697.442379,0.343105,0.577901,0.724736,0.986172
75%,761.478566,0.354078,0.634864,0.810780,1.000000
max,994.808449,0.404527,0.824786,1.000000,1.000000
se,0.323910,0.000074,0.000370,0.000379,0.000129
t,2187.597650,4654.728784,1544.245976,1993.737178,7562.285069


# the market impact model will be 

In [14]:
coeffs = np.array([summary2.ix['mean'],summary.ix['mean']])
coeffs = list(np.mean(coeffs, axis=0))
print coeffs
def MI_model(x):
    "market impact function"
    
    s = np.array(x[0])          #size
    v = np.array(x[1])          #volatility
    P = np.array(x[2])          #POV
        
   
    return coeffs[0]*(s**coeffs[1])*(v **coeffs[2]) * (coeffs[4]*(P **coeffs[3])+ (1-coeffs[4]))


[706.28378945135853, 0.34305784856765598, 0.56587621757055151, 0.75754853087902729, 0.97175923281985044]
